In [1]:
import os
from pathlib import Path
import json
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 800)
pd.set_option('display.width', 1000)

In [2]:
def filter_json(img, ann):
    img_name = img.split(os.sep)[-1]
    img_size = ann.get('size')
    objects = [ format_bbox(obj, img_size) for obj in ann.get('objects') ]
    obj = {
        'name': img_name,
        'width': img_size.get('width'),
        'height': img_size.get('height'),
        'objects': objects,
        'object_count': len(objects),
    }
    return obj


def format_bbox(obj, img_size):
    """ formats (defect detection) points """
    
    bbox = format_bndbox_to_yolo(img_size['width'], img_size['height'],
                                obj['points']['exterior'][0][0],
                                obj['points']['exterior'][0][1],
                                obj['points']['exterior'][1][0],
                                obj['points']['exterior'][1][1],
    )
    
    obj_simp = {
        'classTitle': obj.get('classTitle'),
        'classId': obj.get('classId'),
        'bbox': bbox,
    }
    return obj_simp


def format_bndbox_to_yolo(img_width: float, img_height: float, xmin: float, ymin: float, xmax: float, ymax: float) -> \
        dict[str, float]:
    """ Converts bounding boxes to format: x_center y_center width height """
    width = xmax - xmin
    x_center = xmin + width / 2
    x_center, width = x_center / img_width, width / img_width

    height = ymax - ymin
    y_center = ymin + height / 2
    y_center, height = y_center / img_height, height / img_height

    return { 'x_center': x_center, 'y_center': y_center, 'width': width, 'height': height }


In [37]:
json_label_map = {
    6511999: 0,  # D00, "wheel mark part", "longitudinal crack"
    6512000: 3,  # D10, "equal interval", "transverse crack"
    6512001: 5,  # D20, "partial pavement, overall pavement", "alligator crack"
    6512002: 6,  # D40, "rutting, bump, pothole, separation", "pothole"
    6512003: 7,  # 
}

def get_string_label(obj):
    label_id = json_label_map[ obj['classId'] ]
    key_order = [ 'x_center', 'y_center', 'width', 'height' ]
    int_str = ' '.join([ str(obj['bbox'].get(key)) for key in key_order ])
    return f'{label_id} {int_str}'

In [4]:
def get_img_json(img):
    """ gets image annotations from json as python dict """
    json_fp = get_img_json_filepath(img)
    with open(json_fp, 'r') as f:
        return json.load(f)

def get_img_json_filepath(img):
    return str(img).replace(f'{os.sep}img{os.sep}', f'{os.sep}ann{os.sep}') + '.json'

In [5]:
def get_annotations(dataset_path, limit=None):
    
    if not os.path.exists(dataset_path):
        print('No such dataset: "{}"'.format(dataset_path))
    # dataset_pp_path = f'{SCRATCH}/datasets/RDD2022_allas_pp_{int(args.fraction*100)}'
    
    print('Pre-processing dataset: "{}"'.format(dataset_path))
    # print('Saving preprocessed dataset to: "{}"'.format(dataset_pp_path))
    
    train_img_dir = f'{dataset_path}/train/img'
    train_images = [ img for img in Path(train_img_dir).glob('*.jpg') ]
    train_images = [ img for img in train_images if 'China_Drone' not in str(img) ]
    
    print('Found {:_} images in /train/img'.format(len(train_images)))

    anns = []

    try:
        for idx, img in enumerate(train_images):
            perc = (idx+1)/len(train_images)*100
            print('\rextracting {:_}/{:_} ({:.1f}%): "{}"'.format(idx+1, len(train_images), perc, img.name), end='')
            fn = str(img)
            img_json = get_img_json(fn)
            img_json = filter_json(fn, img_json)
            anns.append(img_json)
            if limit and idx >= limit:
                break
        print()
    except KeyboardInterrupt:
        print('\n... interrupted')
    
    print('done.')
    return anns

In [ ]:
# START: RUN ALL ABOVE
...

In [6]:
# [GENERATE] Generate dataframe from dataset
dataset_path = '../../datasets/RDD2022_allas'
data = get_annotations(dataset_path, limit=None)
df = pd.DataFrame(data).set_index('name')
df

Pre-processing dataset: "../../datasets/RDD2022_allas"
Found 35_984 images in /train/img
extracting 35_984/35_984 (100.0%): "United_States_004804.jpg"
done.


,width,height,objects,object_count
name,,,,
China_MotorBike_000000.jpg,512,512,"[{'classTitle': 'alligator crack', 'classId': ...",3
China_MotorBike_000001.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1
China_MotorBike_000002.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",2
China_MotorBike_000003.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",5
China_MotorBike_000004.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1
...,...,...,...,...
United_States_004800.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",5
United_States_004801.jpg,640,640,"[{'classTitle': 'transverse crack', 'classId':...",1
United_States_004802.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",2


In [7]:
# [SAVE]
df.to_pickle('data/data.pkl')

In [40]:
# [LOAD]
df = pd.read_pickle('data/data.pkl')
df

,width,height,objects,object_count
name,,,,
China_MotorBike_000000.jpg,512,512,"[{'classTitle': 'alligator crack', 'classId': ...",3
China_MotorBike_000001.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1
China_MotorBike_000002.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",2
China_MotorBike_000003.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",5
China_MotorBike_000004.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1
...,...,...,...,...
United_States_004800.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",5
United_States_004801.jpg,640,640,"[{'classTitle': 'transverse crack', 'classId':...",1
United_States_004802.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",2


In [ ]:
# [ANALYSE] Find classes
classes = {}

for idx, (name, data) in enumerate(df.iterrows()):
    print('\r({:_}/{:_})'.format(idx+1, len(df)), end='')
    objects = data['objects']
    for obj in objects:
        tpl = (obj['classId'], obj['classTitle'])
        classes[tpl] = classes.get(tpl, 0) + 1
    # break
print('  done.')

print('\nCOUNT    (ID, TITLE)')
for k, v in classes.items():
    print('{:<8} {}'.format(v, k))

(35_984/35_984)  done.

COUNT    (ID, TITLE)
10324    (6512001, 'alligator crack')
24770    (6511999, 'longitudinal crack')
6458     (6512002, 'pothole')
10612    (6512000, 'transverse crack')
5850     (6512003, 'other corruption')


In [ ]:
# [MODIFY] Apply more processing to df
def get_subset_from_name(name):
    last = name.split('_')[-1]
    return name.replace(f'_{last}', '')

df['subset'] = df.index.map(lambda name: get_subset_from_name(name))
df['labels'] = df['objects'].apply(lambda objects: [get_string_label(obj) for obj in objects] if isinstance(objects, list) else [])
df

,width,height,objects,object_count,subset,labels
name,,,,,,
China_MotorBike_000000.jpg,512,512,"[{'classTitle': 'alligator crack', 'classId': ...",3,China_MotorBike,[5 0.6640625 0.1767578125 0.66796875 0.3496093...
China_MotorBike_000001.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1,China_MotorBike,[6 0.4306640625 0.49609375 0.068359375 0.15625]
China_MotorBike_000002.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",2,China_MotorBike,[0 0.435546875 0.681640625 0.08203125 0.632812...
China_MotorBike_000003.jpg,512,512,"[{'classTitle': 'longitudinal crack', 'classId...",5,China_MotorBike,"[0 0.37890625 0.3095703125 0.125 0.611328125, ..."
China_MotorBike_000004.jpg,512,512,"[{'classTitle': 'pothole', 'classId': 6512002,...",1,China_MotorBike,[6 0.24609375 0.197265625 0.04296875 0.03125]
...,...,...,...,...,...,...
United_States_004800.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",5,United_States,"[0 0.828125 0.8984375 0.203125 0.2, 5 0.46875 ..."
United_States_004801.jpg,640,640,"[{'classTitle': 'transverse crack', 'classId':...",1,United_States,[3 0.36171875 0.78125 0.0859375 0.01875]
United_States_004802.jpg,640,640,"[{'classTitle': 'longitudinal crack', 'classId...",2,United_States,"[0 0.784375 0.859375 0.3 0.26875, 0 0.80859375..."


In [46]:
# [ANALYSE] Find distribution of countries
df.groupby('subset').size()

subset
China_MotorBike     1977
Czech               2829
India               7706
Japan              10506
Norway              8161
United_States       4805
dtype: int64